# Generate FORTRAN static modules with OTI or Multidual algebras.
```
Author: Mauricio Aristizabal
Date last modified: Jan 21 2022
Initial date: Jan 13 2022
Contact: mauriaristi@gmail.com
```

This Notebook lets you generate Fortran modules that implements static versions of the library.

They implement the so called semi-sparse implementation to avoid computational overheads.


In [1]:
import pyoti.core   as coti
from pyoti.fmod_writer import writer

In [13]:
nbases = 3
order  = 3
w = writer(nbases, order, mdual=True)

In [14]:
w.write_file(tab = '  ', is_std_matmul=True)

In [4]:
print(w.write_cr_matrix_form(tab="   ",val_shape = "M"))

FUNCTION MDNUM3_TO_CR_MAT_M(VAL) RESULT(RES)
   IMPLICIT NONE
   TYPE(MDNUM3), INTENT(IN) :: VAL(:,:)
   REAL(DP) :: RES(NUM_IM_DIR*SIZE(VAL,1),NUM_IM_DIR*SIZE(VAL,2)) 
   INTEGER :: NCOLS=1, NROWS=1

   NCOLS = SIZE(VAL,1)
   NROWS = SIZE(VAL,2)

   ! R x R -> R (1, 1)
   RES(1+NROWS*0:NROWS*1,1+NCOLS*0:NCOLS*1) = VAL%R
   ! R x E1 -> E1 (2, 2)
   RES(1+NROWS*1:NROWS*2,1+NCOLS*1:NCOLS*2) = VAL%R
   ! E1 x R -> E1 (2, 1)
   RES(1+NROWS*1:NROWS*2,1+NCOLS*0:NCOLS*1) = VAL%E1
   ! R x E2 -> E2 (3, 3)
   RES(1+NROWS*2:NROWS*3,1+NCOLS*2:NCOLS*3) = VAL%R
   ! E2 x R -> E2 (3, 1)
   RES(1+NROWS*2:NROWS*3,1+NCOLS*0:NCOLS*1) = VAL%E2
   ! R x E3 -> E3 (4, 4)
   RES(1+NROWS*3:NROWS*4,1+NCOLS*3:NCOLS*4) = VAL%R
   ! E3 x R -> E3 (4, 1)
   RES(1+NROWS*3:NROWS*4,1+NCOLS*0:NCOLS*1) = VAL%E3
   ! R x E12 -> E12 (5, 5)
   RES(1+NROWS*4:NROWS*5,1+NCOLS*4:NCOLS*5) = VAL%R
   ! E12 x R -> E12 (5, 1)
   RES(1+NROWS*4:NROWS*5,1+NCOLS*0:NCOLS*1) = VAL%E12
   ! E1 x E2 -> E12 (5, 3)
   RES(1+NROWS*4:NROWS*5,

In [10]:
print( w.write_scalar_function(function_name = "SUM", is_elemental = True, level = 0, 
tab = "    ", f_name = "", lhs_type= "O",lhs_shape='M', rhs_type= "R",rhs_shape='S', 
separator = "*", f_open = "",  f_close = "", 
generator = w.multiplication_like_function_oo, overload = "*" ) )

FUNCTION MDNUMM3N3_SUM_OR_MS(LHS,RHS)&
    RESULT(RES)
    IMPLICIT NONE
    TYPE(MDNUMM3N3), INTENT(IN) :: LHS(:,:)
    REAL(DP), INTENT(IN) :: RHS
    TYPE(MDNUMM3N3) :: RES(SIZE(LHS,1),SIZE(LHS,2)) 

    !  Multiplication like function 'LHS*RHS'

    ! Order 0
    RES%R = LHS%R*RHS%R

    ! Order 1
    RES%E1 = LHS%R*RHS%E1 + LHS%E1*RHS%R
    RES%E2 = LHS%R*RHS%E2 + LHS%E2*RHS%R
    RES%E3 = LHS%R*RHS%E3 + LHS%E3*RHS%R

    ! Order 2
    RES%E12 = LHS%R*RHS%E12 + LHS%E12*RHS%R + LHS%E1*RHS%E2 &
            + LHS%E2*RHS%E1
    RES%E13 = LHS%R*RHS%E13 + LHS%E13*RHS%R + LHS%E1*RHS%E3 &
            + LHS%E3*RHS%E1
    RES%E23 = LHS%R*RHS%E23 + LHS%E23*RHS%R + LHS%E2*RHS%E3 &
            + LHS%E3*RHS%E2

    ! Order 3
    RES%E123 = LHS%R*RHS%E123 + LHS%E123*RHS%R + LHS%E1*RHS%E23 &
             + LHS%E23*RHS%E1 + LHS%E2*RHS%E13 + LHS%E13*RHS%E2 &
             + LHS%E3*RHS%E12 + LHS%E12*RHS%E3

END FUNCTION MDNUMM3N3_SUM_OR_MS



In [3]:
print(w.set_type_constants( level = ""))

! Constant imaginary directions.
! Order 1
TYPE(MDNUMM3N3) :: E1 = MDNUMM3N3(0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E2 = MDNUMM3N3(0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E3 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
! Order 2
TYPE(MDNUMM3N3) :: E12 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E13 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E23 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP)
! Order 3
TYPE(MDNUMM3N3) :: E123 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP)



In [3]:
print(w.write_private(level = "", tab = "  "))

PRIVATE :: DP,DER_R_COS,DER_R_SIN, &
           DER_R_LOG,DER_R_EXP,DER_R_POW



In [3]:
print(w.write_scalar_feval(function_name = "FEVAL", is_elemental = True, level = 0, tab = "    ", 
    f_name = "FUNCTION",  separator = ",", lhs_type= "O", lhs_ptr = False,
    f_open = "(", f_close = ")", addition = " + ", to = False,
    overload = None, write_charact=True ))

ELEMENTAL FUNCTION MDNUMM3N3_FEVAL(X,DERIVS)&
    RESULT(RES)
    IMPLICIT NONE
    !  Definitions
    REAL(DP) :: FACTOR=1, COEF=0.0_DP
    TYPE(MDNUMM3N3), INTENT(IN)  :: X
    TYPE(MDNUMM3N3), INTENT(IN)  :: DERIVS(4)
    TYPE(MDNUMM3N3), INTENT(OUT) :: RES
    TYPE(MDNUMM3N3) :: DX = X, DX_P = X

    !  Set real part of deltas zero.
    DX%R = 0.0_DP
    DX_P%R = 0.0_DP

    ! Sets real part
    RES = DERIVS(1)

    ! Sets order 1
    FACTOR = FACTOR * 1
    COEF = DERIVS(2) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    DX_P = DX_P*DX
    
    ! Sets order 2
    FACTOR = FACTOR * 2
    COEF = DERIVS(3) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    DX_P = DX_P*DX
    
    ! Sets order 3
    FACTOR = FACTOR * 3
    COEF = DERIVS(4) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    
END FUNCTION MDNUMM3N3_FEVAL




In [7]:
print(w.feval_function_o(level = "", lhs_name= "X", deriv_name= "DERIVS", res_name = "RES"  ) ) 

! feval function
RES=0.0_dp
!  Real
RES%R = DERIVS(1)



In [3]:

print(w.gem_like_function_oo( level = "", f_name = "", a_name= "A",
    b_name= "B", c_name= "C", res_name = "RES", separator = "*", 
    f_open = "", f_close = "",  addition = " + " ) ) 

!  General multiplication like function 'A*B + C'

! Order 0
RES%R = C%R + A%R*B%R

! Order 1
RES%E1 = C%E1 + A%R*B%E1 + A%E1*B%R
RES%E2 = C%E2 + A%R*B%E2 + A%E2*B%R
RES%E3 = C%E3 + A%R*B%E3 + A%E3*B%R

! Order 2
RES%E12 = C%E12 + A%R*B%E12 + A%E12*B%R + A%E1*B%E2 &
        + A%E2*B%E1
RES%E13 = C%E13 + A%R*B%E13 + A%E13*B%R + A%E1*B%E3 &
        + A%E3*B%E1
RES%E23 = C%E23 + A%R*B%E23 + A%E23*B%R + A%E2*B%E3 &
        + A%E3*B%E2

! Order 3
RES%E123 = C%E123 + A%R*B%E123 + A%E123*B%R + A%E1*B%E23 &
         + A%E23*B%E1 + A%E2*B%E13 + A%E13*B%E2 &
         + A%E3*B%E12 + A%E12*B%E3



In [4]:
print(w.write_scalar_trivar( function_name = "GEM", is_elemental = True, level = 0, tab = "  ", 
    f_name = "", a_type= "R",  b_type= "O", c_type= "O", 
    separator = "*", f_open = "", f_close = "", addition = " + ",generator = w.gem_like_function_oo, 
    overload = None, write_charact=True ))

ELEMENTAL FUNCTION MDNUMM3N3_GEM_RO(A,B,C)&
  RESULT(RES)
  IMPLICIT NONE
  REAL(DP), INTENT(IN) :: A 
  TYPE(MDNUMM3N3), INTENT(IN) :: B 
  TYPE(MDNUMM3N3), INTENT(IN) :: C 
  TYPE(MDNUMM3N3) :: RES 

  !  General multiplication like function 'A*B + C'

  ! Order 0
  RES%R = C%R + A%R*B%R

  ! Order 1
  RES%E1 = C%E1 + A%R*B%E1 + A%E1*B%R
  RES%E2 = C%E2 + A%R*B%E2 + A%E2*B%R
  RES%E3 = C%E3 + A%R*B%E3 + A%E3*B%R

  ! Order 2
  RES%E12 = C%E12 + A%R*B%E12 + A%E12*B%R + A%E1*B%E2 &
          + A%E2*B%E1
  RES%E13 = C%E13 + A%R*B%E13 + A%E13*B%R + A%E1*B%E3 &
          + A%E3*B%E1
  RES%E23 = C%E23 + A%R*B%E23 + A%E23*B%R + A%E2*B%E3 &
          + A%E3*B%E2

  ! Order 3
  RES%E123 = C%E123 + A%R*B%E123 + A%E123*B%R + A%E1*B%E23 &
           + A%E23*B%E1 + A%E2*B%E13 + A%E13*B%E2 &
           + A%E3*B%E12 + A%E12*B%E3

END FUNCTION MDNUMM3N3_GEM_RO



In [6]:
w.mult_res

[[],
 [[], [], []],
 [[['E1', 'E2'], ['E2', 'E1']],
  [['E1', 'E3'], ['E3', 'E1']],
  [['E2', 'E3'], ['E3', 'E2']]],
 [[['E1', 'E23'],
   ['E23', 'E1'],
   ['E2', 'E13'],
   ['E13', 'E2'],
   ['E3', 'E12'],
   ['E12', 'E3']]]]

In [7]:
w.mult_res_total

[[[[[0, 0, 'R'], [0, 0, 'R']]]],
 [[[[0, 0, 'R'], [0, 1, 'E1']], [[0, 1, 'E1'], [0, 0, 'R']]],
  [[[0, 0, 'R'], [1, 1, 'E2']], [[1, 1, 'E2'], [0, 0, 'R']]],
  [[[0, 0, 'R'], [2, 1, 'E3']], [[2, 1, 'E3'], [0, 0, 'R']]]],
 [[[[0, 0, 'R'], [1, 2, 'E12']],
   [[1, 2, 'E12'], [0, 0, 'R']],
   [[0, 1, 'E1'], [1, 1, 'E2']],
   [[1, 1, 'E2'], [0, 1, 'E1']]],
  [[[0, 0, 'R'], [3, 2, 'E13']],
   [[3, 2, 'E13'], [0, 0, 'R']],
   [[0, 1, 'E1'], [2, 1, 'E3']],
   [[2, 1, 'E3'], [0, 1, 'E1']]],
  [[[0, 0, 'R'], [4, 2, 'E23']],
   [[4, 2, 'E23'], [0, 0, 'R']],
   [[1, 1, 'E2'], [2, 1, 'E3']],
   [[2, 1, 'E3'], [1, 1, 'E2']]]],
 [[[[0, 0, 'R'], [5, 3, 'E123']],
   [[5, 3, 'E123'], [0, 0, 'R']],
   [[0, 1, 'E1'], [4, 2, 'E23']],
   [[4, 2, 'E23'], [0, 1, 'E1']],
   [[1, 1, 'E2'], [3, 2, 'E13']],
   [[3, 2, 'E13'], [1, 1, 'E2']],
   [[2, 1, 'E3'], [1, 2, 'E12']],
   [[1, 2, 'E12'], [2, 1, 'E3']]]]]

In [8]:
w.name_imdir

[['R'], ['E1', 'E2', 'E3'], ['E12', 'E13', 'E23'], ['E123']]

In [3]:
print(w.addition_like_oo(level = "", operator = " + ", lhs_name= "LHS", rhs_name= "RHS", res_name = "RES"))

!  Addition like operation ' + '
!  Real
RES%R = LHS%R + RHS%R
! Order 1
RES%E1 = LHS%E1 + RHS%E1
RES%E2 = LHS%E2 + RHS%E2
RES%E3 = LHS%E3 + RHS%E3
! Order 2
RES%E12 = LHS%E12 + RHS%E12
RES%E13 = LHS%E13 + RHS%E13
RES%E23 = LHS%E23 + RHS%E23
! Order 3
RES%E123 = LHS%E123 + RHS%E123



In [20]:
w.name_imdir

[['R'], ['E1', 'E2'], ['E11', 'E12', 'E22'], ['E111', 'E112', 'E122', 'E222']]